In [4]:
import pandas as pd
import torch

In [5]:
device = "cpu"

In [6]:
model1 = torch.load("model_anger", map_location=device)

2022-11-08 13:12:37.307422: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-08 13:12:37.549162: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-08 13:12:37.636739: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-08 13:12:37.636780: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [7]:
model2 = torch.load("model_sentiment", map_location=device)

In [8]:
model3 = torch.load("model_emotion", map_location=device)

In [9]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [10]:
text = "I am happy"

In [11]:
token = tokenizer.encode_plus(text, return_tensors="pt")

In [12]:
class BERTClass0(torch.nn.Module):
    def __init__(self):
        super(BERTClass0, self).__init__()
        self.model1 = model1
        self.model2 = model2
        self.model3 = model3

    def forward(self, input_ids, attention_masks):

        out1 = self.model1(input_ids=input_ids, attention_mask=attention_masks)
        out2 = self.model2(input_ids=input_ids, attention_mask=attention_masks)
        out3 = self.model3(input_ids=input_ids, attention_mask=attention_masks)

        result = torch.cat([out3["logits"], torch.tensor([[out2["logits"][0][1]]])], 1)

        result[0][0] = out1["logits"][0][0]

        return result

model = BERTClass0()
model.to(device);

In [13]:
output = model(token["input_ids"], token["attention_mask"])
output = output.detach()

In [14]:
torch.sigmoid(output)

tensor([[0.1369, 0.0902, 0.0107, 0.0081, 0.9705, 0.2428, 0.8818, 0.0102, 0.0264,
         0.0369, 0.0855, 0.4204]])

In [15]:
import pandas as pd
col = [ 'anger', 'anticipation', 'disgust', 'fear', 'joy',
       'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust', 'neutral']
pd.DataFrame(torch.sigmoid(output), columns=col)

,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust,neutral
0,0.136933,0.090241,0.010691,0.008126,0.97052,0.242846,0.881821,0.010158,0.026419,0.036877,0.085472,0.420393


In [16]:
device = "cpu"

In [17]:
import torch

In [18]:
_model = torch.load("model_emotion_extra_base", map_location=device)

In [19]:
from torch import nn

In [20]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.roberta = _model
        self.dense_layer_1 = nn.Linear(768+110+110, 256)
        self.dropout = nn.Dropout(0.2)
        self.dense_layer_2 = nn.Linear(256, 128)
        self.dropout_2 = nn.Dropout(0.2)
        self.cls_layer = nn.Linear(128, 12, bias = True)

    def forward(self, input_ids, attention_masks, feature1, feature2):

        pooled_output = self.roberta(input_ids=input_ids, attention_mask=attention_masks)["pooler_output"]

        concat = torch.cat( (pooled_output, feature1, feature2), dim=1 )

        x = self.dense_layer_1(concat)
        x = self.dropout(x)
        x_1 = self.dense_layer_2(x)
        x_2 = self.dropout_2(x_1)

        logits = self.cls_layer(x_2)

        return logits

model4 = BERTClass()

In [21]:
model_ = torch.load("model_emotion_extra", map_location=device)

In [22]:
model4.load_state_dict(model_.state_dict())

<All keys matched successfully>

In [23]:
class BERTClass1(nn.Module):
    def __init__(self):
        super().__init__()
        self.model1 = model1
        self.model2 = model2
        self.model4 = model4

    def forward(self, input_ids, attention_masks, _feature1, _feature2):

        out1 = self.model1(input_ids=input_ids, attention_mask=attention_masks)
        out2 = self.model2(input_ids=input_ids, attention_mask=attention_masks)
        out3 = self.model4(input_ids=input_ids, attention_masks=attention_masks, feature1=_feature1, feature2=_feature2)

        result = out3

        result[0][0] = out1["logits"][0][0]
        result[0][11] = out2["logits"][0][1]

        return result

model = BERTClass1()

In [24]:
text = "I am Happy"
token = tokenizer.encode_plus(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length=150,
            padding='max_length',
            return_token_type_ids=True,
            return_tensors="pt"
        )

Ignored unknown kwarg option direction


In [25]:
import spacy

2022-11-08 13:13:55.594853: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-08 13:13:55.595106: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-08 13:13:55.595120: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Linux): /proc/driver/nvidia/version does not exist


In [26]:
nlp = spacy.load("en_core_web_sm")

In [27]:
def give_dep_features(sentence):
  sent_dep_list =[]
  doc = nlp(sentence)
  for _token in doc:
    sent_dep_list.append(str(_token.dep_))

  return sent_dep_list

In [28]:
feature1 = give_dep_features(text)
feature1

['nsubj', 'ROOT', 'acomp']

In [29]:
def give_noun_features(sentence):
  sent_dep_list =[]
  doc = nlp(sentence)
  for _token in doc:
    sent_dep_list.append(str(_token.head.pos_))

  return sent_dep_list

In [30]:
feature2 = give_noun_features(text)
feature2

['AUX', 'AUX', 'AUX']

In [31]:
noun_dictionary = {'VERB': 1, 'NOUN': 2, 'AUX': 3, 'ADP': 4, 'PROPN': 5, 'ADJ': 6, 'SYM': 7, 'ADV': 8, 'PRON': 9, 'NUM': 10, 'PUNCT': 11, 'INTJ': 12, 'X': 13, 'SCONJ': 14, 'SPACE': 15, 'PART': 16, 'DET': 17, 'CCONJ': 18}

In [32]:
dependency_dictionary = {'nsubj': 1, 'punct': 2, 'ROOT': 3, 'prep': 4, 'dobj': 5, 'pobj': 6, 'advmod': 7, 'det': 8, 'amod': 9, 'aux': 10, 'compound': 11, 'conj': 12, 'ccomp': 13, 'cc': 14, 'dep': 15, 'nmod': 16, 'poss': 17, 'npadvmod': 18, 'advcl': 19, 'acomp': 20, 'neg': 21, 'attr': 22, 'xcomp': 23, 'appos': 24, 'mark': 25, 'relcl': 26, 'nummod': 27, 'prt': 28, 'auxpass': 29, 'intj': 30, 'pcomp': 31, 'acl': 32, 'nsubjpass': 33, 'case': 34, 'dative': 35, 'quantmod': 36, 'csubj': 37, 'oprd': 38, 'predet': 39, 'agent': 40, 'expl': 41, 'parataxis': 42, 'meta': 43, 'preconj': 44, 'csubjpass': 45}

In [33]:
def give_padding(sent):

  var_len = len(sent)
  padding_len = 110-var_len
  padding = [0]*padding_len
  sent = sent + padding

  return sent[:110]

def give_encoded_noun_list(sent):
  encoded_list = []

  for i in sent:
    encoded_list.append(noun_dictionary[i])

  return give_padding(encoded_list)

def give_encoded_dependency_list(sent):
  encoded_list = []

  for i in sent:
    encoded_list.append(dependency_dictionary[i])

  return give_padding(encoded_list)

In [34]:
feature1 = give_encoded_dependency_list(feature1)

In [35]:
feature2 = give_encoded_noun_list(feature2)

In [36]:
result = model(token["input_ids"], token["attention_mask"], torch.tensor(feature1, dtype=torch.long).unsqueeze(0), torch.tensor(feature2, dtype=torch.long).unsqueeze(0))

In [37]:
import pandas as pd
col = [ 'anger', 'anticipation', 'disgust', 'fear', 'joy',
       'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust', 'neutral']
pd.DataFrame(torch.sigmoid(result.detach()), columns=col)

,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust,neutral
0,0.156564,0.00001,2.350510e-10,0.000016,1.0,0.201354,0.992397,7.836081e-11,0.000054,0.000015,0.000051,0.436435
